In [2]:
import dhlab as dh
import dhlab.api.dhlab_api as api
import pandas as pd
from dhlab.nbtokenizer import tokenize
import jsonlines
dh.css()

# Lemmatize a set

a = dh.totals(100000)
lemma = dh.WordLemma(list(a.index)).lemmas
reduced_set = pd.DataFrame(lemma.merge(a.reset_index(), left_on=0, right_on='index').groupby(2)['freq'].max())
lower_set = pd.DataFrame(reduced_set.groupby(lambda x: x.lower())['freq'].sum())
lower_set.sort_values(by='freq', ascending=False)

# Generate sentences

In [41]:
a = api.word_concordance(dhlabid=list(corpus.frame.sample(10).dhlabid), words = ['arbeid'])

In [4]:
def show_concs(a):
    res = []
    for i, x in a.iterrows():
        try:
            res.append(' '.join([x['before'], x['target'], x['after']]))
        except:
            pass
    return res

In [52]:
#show_concs(a)

In [32]:
def generate_word_sequences_from_corpus_word(corpus, samples, words = None, before = 12, after = 12, limit=1000):
    sentences = api.word_concordance(dhlabid=list(corpus.sample(samples).dhlabid), words = words, before = before, after = after, limit = limit)
    #a = sentences.show(style=False, n=limit).concordance.apply(lambda x:x.replace('<b>','').replace('</b>','')).values
    return sentences

In [66]:
def generate_word_sequences_from_corpus_wkonk(corpus, samples, words = None, limit=1000):
    sentences = api.word_concordance(dhlabid=list(corpus.sample(samples).dhlabid), words = words, before = 24, after = 0, limit = limit)

    return show_concs(sentences)

In [15]:
def convert(sentence, lower=False):
    """Tokenize i a string, turn all tokens to lower case and 
    remove punctuations and other stuff not normally part of a word"""
    lst = tokenize(sentence)
    lowered = [x for x in lst if x.isalpha()]
    return lowered

## Select corpus

In [5]:
corpus = dh.Corpus(doctype='digavis',  from_year=1978, to_year=1984, limit = 25)


In [6]:
len(corpus.frame)

25

In [33]:
s = generate_word_sequences_from_corpus_word(corpus.frame, 5, words = ['og'], before =0, after=24, limit = 10)

In [41]:
s.after

0     for mange ble vel lørdagens Wenche Myhre-show ...
1     etter endt show skrev hun autografer i nesten ...
2     spilte , åtte lokale grupper / personer var på...
3     fargesprakende fra første til siste stund , og...
4     akkurat like flott som vi hadde trodd . Wenche...
5     opplagt , og ga alt i den , times lange sangop...
6     ga alt i den , times lange sangopptrede- nen ....
7     pensjonister , da disse gikk gratis . Det hele...
8                  høyttalere var plassert rundt i hele
9     fint repertoar , men toppen på det hele ble nå...
10    barnehageaksjonen i bydelen har kommet med. De...
11    en bebudet demon- strasjon I Furuset bydel , v...
12    Kurland , som hittil har vært disponert av per...
13    formannskapet har forlengst forkastet forslage...
14    penger avsatt til bygging . Tilgrensende bøore...
15    bygging settes I gang . AKTUELL OPPGAVE Dette ...
16    finan » sutvalgets vedtak om at det skal sette...
17    kjempe for åpning av de nye barnehagene . 

In [38]:
str(s.before) + ' ' + str(s.target) + ' ' + str(s.after)

"0     None\n1     None\n2     None\n3     None\n4     None\n5     None\n6     None\n7     None\n8     None\n9     None\n10    None\n11    None\n12    None\n13    None\n14    None\n15    None\n16    None\n17    None\n18    None\n19    None\n20    None\n21    None\n22    None\n23    None\n24    None\n25    None\n26    None\n27    None\n28    None\n29    None\n30    None\n31    None\n32    None\n33    None\n34    None\n35    None\n36    None\n37    None\n38    None\n39    None\n40    None\n41    None\n42    None\n43    None\n44    None\n45    None\n46    None\n47    None\n48    None\n49    None\nName: before, dtype: object 0     og\n1     og\n2     og\n3     og\n4     og\n5     og\n6     og\n7     og\n8     og\n9     og\n10    og\n11    og\n12    og\n13    og\n14    og\n15    og\n16    og\n17    og\n18    og\n19    og\n20    og\n21    og\n22    og\n23    og\n24    og\n25    og\n26    og\n27    og\n28    og\n29    og\n30    og\n31    og\n32    og\n33    og\n34    og\n35    og\n36    og\n3

In [63]:
aften = corpus.frame[corpus.frame.title.str.contains('aftenposten')]
len(aften)

4434

## Generate json lines

%%time
for i in range(400):
    if i%100 == 0:
        print(i)
    a = generate_word_sequences_from_corpus(aften, 100, words = ['Alta-saken'], limit=1000)
    with open(f"extract_digavis_aftenpost_78_84_Alta.txt",mode='a') as wr:
        for sent in map(convert, a):
            wr.write(' '.join(sent))
            wr.write('\n')
    

In [67]:
%%time
with open(f"extract_digavis_aftenpost_78_84_Alta.txt",mode='w') as wr:
    for i in range(4):
        if i%100 == 0:
            print(i)
        a = generate_word_sequences_from_corpus_wkonk(aften, 100, words = ['.'], limit=1000)
        for sent in a:
            wr.write(sent)
            wr.write('\n')
    

0
CPU times: user 2.67 s, sys: 190 ms, total: 2.86 s
Wall time: 1min 17s
